In [12]:
# Andrej Karpathy
# https://www.youtube.com/watch?v=kCc8FmEb1nY&t=31s
# b站有中英字幕 
# https://www.bilibili.com/video/BV1CP41147Cw

# 获取 tiny shakespeare数据文件，本代码仓库存放位置为dataset目录中
# Linux
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# Windows PowerShell
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -Outfile input.txt

In [13]:
from xml.etree.ElementPath import xpath_tokenizer

with open('pretrain/dataset/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [14]:
print(len(text))

1115394


In [15]:
print(text[0:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [16]:
# 统计字符数，作为我们的词本
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)  # 意味着我们的token种类数量为65
print(''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [17]:
# 实现一个encode、decode
# mapping 映射，最简单的tokenizer
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder 输入字符串，转化成数字列表
decode = lambda l: ''.join([itos[i] for i in l])  # decoder 输入数字列表，转化成字符串

print(encode("hello world,i am qingchen"))
print(decode(encode("hello world,i am qingchen")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 6, 47, 1, 39, 51, 1, 55, 47, 52, 45, 41, 46, 43, 52]
hello world,i am qingchen


In [18]:
# 我们造了一个最简单的tokenizer，character-level tokenizer，词本很短，编码和解码最简单， 
# 像Sentence-Piece、tiktoken等，采用子词（sub-word-level）级别。
# 简单的tokenizer导致我们输出向量会很长，比如相同的字符串'hello'，我们转换后是5个[ 21, 23, 2, 3, 4]，而且其他的tokenizer可能就两个[ 23.2, 88.9]

使用pytorch  
-- 我是windows10系统同时带nvida显卡

In [19]:
# 建议安装anaconda，然后去NVIDIA安装CUDA，最后安装pytorch

# windows下anaconda和CUDA安装都很简单
# anaconda
# https://www.anaconda.com/download

# CUDA
# https://developer.nvidia.com/cuda-toolkit

# 安装完后，打开cmd或者powershell创建一个虚拟环境 

# !conda create --name pytorch python=3.11.8

# 安装pytorch 可以直接访问 https://pytorch.org/get-started/locally/#windows-python 这里提供了一个根据你的操作系统和有无GPU选择来生成安装命令

# !conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia

# jupyter notebook （anaconda安装自带notebook）
# 下面这条命令创建一个kernel核心来选择我们刚刚安装了pytorch的虚拟环境

# !python -m ipykernel install --name pytorch --display-name "pytorch"

# 或者此种方式
# conda install nb_conda_kernels

In [20]:
# 测试torch安装
import torch

x = torch.rand(5, 3)
print(x)

tensor([[0.9062, 0.2759, 0.0258],
        [0.2173, 0.6806, 0.4048],
        [0.0426, 0.4487, 0.3127],
        [0.8835, 0.2861, 0.2647],
        [0.6966, 0.1744, 0.4185]])


In [21]:
print(torch.cuda.is_available())
print(torch.backends.cudnn.is_available())
print(torch.cuda_version)
print(torch.backends.cudnn.version())


True
True
12.4
90100


In [22]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [23]:
# 训练集和测试集
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [24]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [25]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
    context = x[:i + 1]
    target = y[i]
    print(f"当输入是{context}时，目标值是{target}")

当输入是tensor([18])时，目标值是47
当输入是tensor([18, 47])时，目标值是56
当输入是tensor([18, 47, 56])时，目标值是57
当输入是tensor([18, 47, 56, 57])时，目标值是58
当输入是tensor([18, 47, 56, 57, 58])时，目标值是1
当输入是tensor([18, 47, 56, 57, 58,  1])时，目标值是15
当输入是tensor([18, 47, 56, 57, 58,  1, 15])时，目标值是47
当输入是tensor([18, 47, 56, 57, 58,  1, 15, 47])时，目标值是58


In [26]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [27]:
print("---------")

for b in range(batch_size):  # 并行处理维度
    for i in range(block_size):  # 时间维度 ？ 原视频写的是：time dimension，但是感觉就是 最大上下文context length 或者输入的最长token数量
        context = xb[b, :i + 1]
        target = yb[b, i]
        print(f"当输入是：{context.tolist()}，目标值为：{target}")

---------
当输入是：[24]，目标值为：43
当输入是：[24, 43]，目标值为：58
当输入是：[24, 43, 58]，目标值为：5
当输入是：[24, 43, 58, 5]，目标值为：57
当输入是：[24, 43, 58, 5, 57]，目标值为：1
当输入是：[24, 43, 58, 5, 57, 1]，目标值为：46
当输入是：[24, 43, 58, 5, 57, 1, 46]，目标值为：43
当输入是：[24, 43, 58, 5, 57, 1, 46, 43]，目标值为：39
当输入是：[44]，目标值为：53
当输入是：[44, 53]，目标值为：56
当输入是：[44, 53, 56]，目标值为：1
当输入是：[44, 53, 56, 1]，目标值为：58
当输入是：[44, 53, 56, 1, 58]，目标值为：46
当输入是：[44, 53, 56, 1, 58, 46]，目标值为：39
当输入是：[44, 53, 56, 1, 58, 46, 39]，目标值为：58
当输入是：[44, 53, 56, 1, 58, 46, 39, 58]，目标值为：1
当输入是：[52]，目标值为：58
当输入是：[52, 58]，目标值为：1
当输入是：[52, 58, 1]，目标值为：58
当输入是：[52, 58, 1, 58]，目标值为：46
当输入是：[52, 58, 1, 58, 46]，目标值为：39
当输入是：[52, 58, 1, 58, 46, 39]，目标值为：58
当输入是：[52, 58, 1, 58, 46, 39, 58]，目标值为：1
当输入是：[52, 58, 1, 58, 46, 39, 58, 1]，目标值为：46
当输入是：[25]，目标值为：17
当输入是：[25, 17]，目标值为：27
当输入是：[25, 17, 27]，目标值为：10
当输入是：[25, 17, 27, 10]，目标值为：0
当输入是：[25, 17, 27, 10, 0]，目标值为：21
当输入是：[25, 17, 27, 10, 0, 21]，目标值为：1
当输入是：[25, 17, 27, 10, 0, 21, 1]，目标值为：54
当输入是：[25, 17, 27, 10, 0, 21, 1, 54]，目标值为：39


接下来构建一个模型

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # 构造函数 创建一个词表大小为 65x65
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # 前向
    def forward(self, idx, targets=None):
        # 将输入的idx从 4x8 变成 4x8x65
        logits = self.token_embedding_table(idx)  # (B,T,C) B: Batch size 这里是4；T: context length 这里是8；C：vocab_size 65

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            # 交叉熵
            loss = F.cross_entropy(logits, targets)  # 这里不改变张量结构会报错

        return logits, loss

    def generate(self, idx, max_new_token):
        # idx 是当前上下文中大小为 (B,T) 的索引数组
        for _ in range(max_new_token):
            # 获取预测值
            logits, loss = self(idx)
            # 专注于最后一步
            logits = logits[:, -1, :]  # 变成(B,C)
            # 加入softmax函数得到概率，dim=-1取最后一个维度来计算也就是C，65
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution 从分布中抽样
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx


m = BigramLanguageModel(vocab_size=vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [29]:
# 切片作用
# tensor = torch.randint(0, 5, (4, 8, 4))
tensor = torch.tensor([[[0, 1, 2, 3],
                        [4, 5, 6, 7],
                        [8, 9, 10, 11],
                        [12, 13, 14, 15]],

                       [[16, 17, 18, 19],
                        [20, 21, 22, 23],
                        [24, 25, 26, 27],
                        [28, 29, 30, 31]],

                       [[32, 33, 34, 35],
                        [36, 37, 38, 39],
                        [40, 41, 42, 43],
                        [44, 45, 46, 47]],

                       [[48, 49, 50, 51],
                        [52, 53, 54, 55],
                        [56, 57, 58, 59],
                        [60, 61, 62, 63]]])

tensor1 = tensor[:, -1, :]
print(tensor1.shape)
print(tensor1)

tensor2 = tensor[:, -1:, :]
print(tensor2.shape)
print(tensor2)

torch.Size([4, 4])
tensor([[12, 13, 14, 15],
        [28, 29, 30, 31],
        [44, 45, 46, 47],
        [60, 61, 62, 63]])
torch.Size([4, 1, 4])
tensor([[[12, 13, 14, 15]],

        [[28, 29, 30, 31]],

        [[44, 45, 46, 47]],

        [[60, 61, 62, 63]]])


In [30]:
# 优化模型

optimizer = torch.optim.Adam(m.parameters(), lr=0.001)
# torch.optim.SGD(m.parameters(), lr=0.001) 随机梯度下降

batch_size = 32
for epoch in range(20000):
    xb, yb = get_batch('train')

    # 计算损失
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4846878051757812


In [31]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=400)[0].tolist()))





QUCatheesicis bas, angomitouze chifomerawhe ty ag

Doone thesellande west, me fffol, whandiffe IAfithods misue, knild he hthe hicofudirer' My ayosbly louroura s mu, uthos s reveprthoukerdi't avorure fotemowe.
Whamo es t, tstt g t RTRushy,
WAsbr spr my ou pl y,
Witoft at o s me,
Whabrethe Cicuomants awonte qungur thme wrar d parsupl by:
'sul ve ave,
Kconit ped bim; fam elathelch easutlll teye A 


#### The Mathematical trick in self-attention 自注意力的数学窍门

In [1]:
import torch

# consider the following toy example：
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch ,time , channels
x = torch.randn(B, T, C)
x.shape


torch.Size([4, 8, 2])

In [73]:
# version 1
# we want x[b,t] = mean_{i<=t} x[b,i] 要前面的平均值
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]
        xbow[b, t] = torch.mean(xprev, 0)

In [59]:
for b in range(B):
    print(b)
    print(x[b, :3])
    print(x[b, :3].shape)
    print(torch.mean(x[b, :3], 0))

0
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255]])
torch.Size([3, 2])
tensor([ 0.1490, -0.3199])
1
tensor([[ 1.3488, -0.1396],
        [ 0.2858,  0.9651],
        [-2.0371,  0.4931]])
torch.Size([3, 2])
tensor([-0.1342,  0.4395])
2
tensor([[-0.6631, -0.2513],
        [ 1.0101,  0.1215],
        [ 0.1584,  1.1340]])
torch.Size([3, 2])
tensor([0.1685, 0.3348])
3
tensor([[ 1.6455, -0.8030],
        [ 1.3514, -0.2759],
        [-1.5108,  2.1048]])
torch.Size([3, 2])
tensor([0.4954, 0.3420])


In [74]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [75]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [76]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [63]:
# 使用矩阵乘法完成上面的操作
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b  # 矩阵乘法
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [64]:
a = torch.tril(torch.ones(3, 3))  # 求累加
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [65]:
a = torch.tril(torch.ones(3, 3))  # 累加
a = a / torch.sum(a, 1, keepdim=True)  # 平均
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [77]:
# version 2
# 实现一开始的两个循环
wei = torch.tril(torch.ones(T, T))  # 累加
wei = wei / wei.sum(1, keepdim=True)  # 平均
xbow2 = wei @ x
print(torch.allclose(xbow2, xbow2))
print(wei.shape)
print(wei)
print(xbow2.shape)
print(xbow2)

True
torch.Size([8, 8])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 

In [78]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

加权求和


In [88]:
# version 3
# softmax
tril = torch.tril(torch.ones(T, T))
wei3 = torch.ones(T, T)
wei3 = wei3.masked_fill(tril == 0, float('-inf'))
print(wei3)
wei3 = F.softmax(wei3, dim=-1)
print(wei3.shape)
print(wei3)
xbow3 = wei3 @ x
torch.allclose(xbow2, xbow3)

tensor([[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
torch.Size([8, 8])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.125

True

In [87]:
xbow[0], xbow3[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [22]:
# version4 self attention !!!
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)
B, T, C = 4, 8, 32  # BATCH , TIME , CHANNELS
x = torch.randn(B, T, C)

# see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)  # 我有什么
query = nn.Linear(C, head_size, bias=False)  # 我感兴趣什么
value = nn.Linear(C, head_size, bias=False)  # 信息聚合
k = key(x)  # (B,T,16)
q = query(x)  # (B,T,16)
wei = q @ k.transpose(-2, -1)  # (B,T,16) @ (B,16,T) -->(B,T,T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)

out = wei @ v
out1 = wei @ x
print(out.shape)
print(out1.shape)

torch.Size([4, 8, 16])
torch.Size([4, 8, 32])


In [7]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [15]:
# scale attention 注意力缩放
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5

In [16]:
k.var()

tensor(1.0966)

In [17]:
q.var()

tensor(0.9416)

In [18]:
wei.var()

tensor(1.0065)

In [19]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [21]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [29]:
idx = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
idx.shape

torch.Size([2, 10])

In [31]:
idx[:,-2:]

tensor([[8, 9],
        [8, 9]])